Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [1]:
import numpy as np

def apk(actual, predicted, k=12):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
def precision(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]

    correct_predictions = [p for p in predicted if p in actual]

    return len(correct_predictions) / len(predicted)

def recall(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]

    correct_predictions = [p for p in predicted if p in actual]

    return len(correct_predictions) / len(actual)

def precision12(actual, predicted, k=12):
    return np.mean([precision(a,p,k) for a,p in zip(actual, predicted)])

def recall12(actual, predicted, k=12):
    return np.mean([recall(a,p,k) for a,p in zip(actual, predicted)])

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [4]:
import pandas as pd

# Read input files

In [5]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

CPU times: user 3.68 s, sys: 3.08 s, total: 6.76 s
Wall time: 2.92 s


In [6]:
val = transactions[transactions.week == transactions.week.max()-3]

# Testing week or latest week for submission

In [7]:
Testting = True
test_week = transactions.week.max()
transactions = transactions[transactions.week > transactions.week.max() - 10]
transactions = transactions[transactions.week < transactions.week.max()]

# Testting = False
# test_week = transactions.week.max()+1
# transactions = transactions[transactions.week > transactions.week.max() - 9]


# Generating candidates

### Last purchase candidates

In [8]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 31.6 s, sys: 434 ms, total: 32.1 s
Wall time: 31.8 s


In [9]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
92,2020-06-24,2020-06-30
93,2020-07-01,2020-07-07
94,2020-07-08,2020-07-14
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25


In [10]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 1.12 s, sys: 54 ms, total: 1.18 s
Wall time: 1.18 s


In [11]:
candidates_last_purchase = transactions.copy()

In [12]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])

candidates_last_purchase.week=weeks

CPU times: user 45.4 s, sys: 247 ms, total: 45.7 s
Wall time: 45.3 s


# Add my own implementation of candidate generation

In [13]:
# Generating candidates based on repurchasing
repurchase_candidates = transactions.copy()
repurchase_candidates['week'] += 1 

# Merge repurchase candidates with customer information
repurchase_candidates = pd.merge(repurchase_candidates, customers, on='customer_id', how='left')
repurchase_candidates = pd.merge(repurchase_candidates, articles, on='article_id', how='left')

# Drop unnecessary columns
repurchase_candidates.drop(['price', 'sales_channel_id', 'year', 'month'], axis=1, inplace=True)

# Rename columns to match the rest of the data
repurchase_candidates.rename(columns={'week': 'previous_purchase_week'}, inplace=True)

# Merge repurchase candidates with the existing last purchase candidates
candidates_last_purchase = pd.merge(
    candidates_last_purchase,
    repurchase_candidates[['customer_id', 'article_id', 'previous_purchase_week']], 
    left_on=['customer_id', 'article_id', 'week'], 
    right_on=['customer_id', 'article_id', 'previous_purchase_week'], 
    how='left'
)
# Fill NaN values in 'previous_purchase_week' with a value that represents no repurchase
candidates_last_purchase['previous_purchase_week'].fillna(-1, inplace=True)

# Create a new column 'repurchased' indicating repurchase
candidates_last_purchase['repurchased'] = (candidates_last_purchase['previous_purchase_week'] != -1).astype(int)

# Drop the 'previous_purchase_week' column as it's no longer needed
candidates_last_purchase.drop('previous_purchase_week', axis=1, inplace=True)




### Bestsellers candidates

In [14]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1
bestsellers_previous_week.week
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
36,96,760084003,1,0.025094
37,96,866731001,2,0.024919
38,96,600886001,3,0.022980
39,96,706016001,4,0.033197
40,96,372860002,5,0.013193
41,96,610776002,6,0.008318
42,96,877278002,7,0.025036
43,96,547780003,8,0.024814
44,96,817354001,9,0.021913
45,96,827968001,10,0.016436


**Use either the above one or my own implmentation below**

In [15]:
# # Calculate the number of unique customers who bought each article
# unique_customers_per_article = transactions.groupby('article_id')['customer_id'].nunique().reset_index()
# unique_customers_per_article.rename(columns={'customer_id': 'unique_customers'}, inplace=True)

# # Rank articles based on the number of unique customers
# bestsellers_ranked = unique_customers_per_article.sort_values('unique_customers', ascending=False).reset_index(drop=True)
# bestsellers_ranked['bestseller_rank'] = bestsellers_ranked.index + 1

# # Merge the bestseller ranks with the articles information
# bestsellers_previous_week = pd.merge(bestsellers_ranked[['article_id', 'bestseller_rank']], articles, on='article_id')

# # Calculate mean price for each article in each week
# mean_price = transactions.groupby(['week', 'article_id'])['price'].mean().reset_index()

# # Identify the top 12 bestsellers for each week based on mean price
# bestsellers_previous_week = mean_price.groupby('week', group_keys=False).apply(
#     lambda x: x.sort_values('price', ascending=False).head(12)
# ).reset_index(drop=True)


# bestsellers_previous_week['bestseller_rank'] = bestsellers_previous_week.groupby('week').cumcount() + 1
# bestsellers_previous_week = pd.merge(bestsellers_previous_week, articles, on='article_id')

# # Adjust week for the next week
# bestsellers_previous_week['week'] += 1


In [16]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [17]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,season,year,month,week
27958681,2020-06-24,1456826891333599,829147001,0.025407,2,2,-28,6,92
27923763,2020-06-24,2231838139078336,865917001,0.010153,1,2,-28,6,92
27897727,2020-06-24,3204013276717965,854637001,0.022017,1,2,-28,6,92
27938842,2020-06-24,3427046018491089,837278001,0.050831,2,2,-28,6,92
27944066,2020-06-24,3725161734497764,501620026,0.015237,2,2,-28,6,92
...,...,...,...,...,...,...,...,...,...
30728907,2020-08-25,18437406193254500389,717490071,0.008458,1,2,-28,8,100
30704649,2020-08-25,18440617515054925735,892558004,0.016932,1,2,-28,8,100
30703747,2020-08-25,18444212647743429500,578476001,0.050831,2,2,-28,8,100
30734878,2020-08-25,18445310347432716808,784415002,0.015237,1,2,-28,8,100


In [18]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [19]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [20]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [21]:
candidates_bestsellers_test_week.columns

Index(['t_dat', 'customer_id', 'sales_channel_id', 'season', 'year', 'month',
       'week', 'article_id', 'bestseller_rank', 'price'],
      dtype='object')

In [22]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [23]:
candidates_bestsellers.columns

Index(['t_dat', 'customer_id', 'sales_channel_id', 'season', 'year', 'month',
       'week', 'article_id', 'price'],
      dtype='object')

# Combining transactions and candidates / negative examples

In [24]:
transactions['purchased'] = 1

In [25]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [26]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [27]:
data.purchased.mean()

0.14557263505069387

### Add bestseller information

In [28]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [29]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [30]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [31]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [32]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [33]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [34]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank',
'buys_womens_clothing', 'buys_mens_clothes', 'buys_kid_baby_clothes']
#  'year', 'month',

In [35]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 430 ms, sys: 272 ms, total: 702 ms
Wall time: 702 ms


# Model training

In [36]:
from lightgbm.sklearn import LGBMRanker
from tqdm import tqdm


In [37]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [38]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.919451
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.270046
[LightGBM] [Debug] init for col-wise cost 0.282371 seconds, init for row-wise cost 2.885199 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.744472 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1148
[LightGBM] [Info] Number of data points in the train set: 10613878, number of used features: 21
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
CPU times: user 27.6 s, sys: 3.15 s, total: 30.7 s
Wall time: 11.7 s


In [39]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9986302597956186
buys_kid_baby_clothes 0.0005111906938349536
article_id 0.00024768523690523624
age 0.0001898085423966064
buys_mens_clothes 0.00016244416566143965
club_member_status 5.3298651294931164e-05
postal_code 4.028164354188945e-05
Active 3.487301348381588e-05
section_no 2.8606792046756303e-05
index_group_no 2.805783284855256e-05
garment_group_no 1.8795895231658334e-05
product_type_no 1.734239890136676e-05
department_no 1.5046590405606076e-05
colour_group_code 1.1603491210183437e-05
fashion_news_frequency 1.0705256618462362e-05
FN 0.0
index_code 0.0
perceived_colour_master_id 0.0
perceived_colour_value_id 0.0
graphical_appearance_no 0.0
buys_womens_clothing 0.0


# Calculate predictions

In [40]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 12.2 µs


# Create submission

In [41]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [42]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 7.17 s, sys: 255 ms, total: 7.42 s
Wall time: 7.42 s


In [43]:
if Testting:
    positive_items_val = val.groupby(['customer_id'])['article_id'].apply(list)
    # creating validation set for metrics use case
    val_users = positive_items_val.keys()
    val_items = []
    for i,user in tqdm(enumerate(val_users)):
        val_items.append(positive_items_val[user])

    print("Total users in validation:", len(val_users))
    print("mAP12 Score on Validation set:", mapk(val_items, preds))
    print("precision Score on Validation set:", precision12(val_items, preds))
    print("recall Score on Validation set:", recall12(val_items, preds))

80253it [00:00, 144161.21it/s]


Total users in validation: 80253
mAP12 Score on Validation set: 0.005468208923134587
precision Score on Validation set: 0.004916742468609688
recall Score on Validation set: 0.020868452057663366


In [44]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [45]:
sub_name = 'candidateRadekRepurchase_oldBestseller'
sub.to_csv(f'{sub_name}.csv.gz', index=False)